In [1]:
#===================================================================IMPORTS
import numpy as np, scipy.stats as st
import scipy.stats
import random
import numpy as np

#===================================================================PARAMETERS
warmup = 30
runs = int(526)  # Number of Replications 69
percentage_traditional = float(0.30)
percentage_crowdshipping = float(0.20)
percentage_parcel_locker = float(0.50)
debug = "off"
results = "on"
mean_results = "on"
RANDOM_STATE = 31415




#===================================================================FUNCTIONS
def input_csv():
    import csv
    f = open('2021-02-20-frequencias sem outlines-Copy1.csv')
    # f = open('prazos de entrega - expressa e normal.csv')
    texpr = []
    tnorm = []
    csv_f = csv.reader(f, delimiter=';')
    i = 0
    for row in csv_f:
        if i > 0:
            texpr.append(int(row[0]))
            tnorm.append(int(row[1]))
        i = i + 1

    return texpr, tnorm


def input_csv_mix():
    import csv
    f = open('2021-02-20-frequencias sem outlines-Copy1.csv')
    # f = open('prazos de entrega - expressa e normal.csv')
    t = []
    csv_f = csv.reader(f, delimiter=';')
    i = 0
    for row in csv_f:
        if i > 0:
            t.append(int(row[0]))
            t.append(int(row[1]))
        i = i + 1

    return t


def monte_carlo(data):
    results, edges = np.histogram(data, density=True)
    p = []
    sum = 0
    i = 0
    for i in range(len(results)):
        sum = sum + results[i]
        if sum >= 1:
            sum = 1
        p.append(sum)
    x = random.random()
    i = 0
    for i in range(len(p)):
        if x <= p[i]:
            return int(edges[i])
    return int(edges[i])

def mean_confidence_interval_and_h(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
    return m, m - h, m + h, h

# Mean Test
replication_km = []
rep_km = []
    
    

#===================================================================REPLICATION BEGINS
for l in range(runs):
    
    #Replication Parameters
    day_counter = int(1)
    max_vehicles = int(0)
    min_vehicles = int(100000)
    overall_mileage = int(0)
    overall_packages = int(0)
    cl = 9
    total_days = warmup + 90  # Number of days
    days = total_days
    


    # Crowdshipping
        
    km_crowdshipping_total = float(0)
    crowdshipping_deliveries_total = int(0) 
    total_crowdshipping_vehicles = int(0)
    
    # parcel locker
    km_parcelocker_total = float(0)
    parcelocker_deliveries_total = int(0)

    tw = []

    for t in range(25):
        tw.append(0)

#===================================================================DAILY LOOP
        
    while days > 0:

        resp1 = int(np.random.normal(loc=98, scale=4, size=None))  # Normal Distribuition
        orders_crowdshipping = int(0)
        orders_parcelocker = int(0)

        

                                    # crowdshipping
        km_crowdshipping = []
        crowdshipping_deliveries = []
        crowdshipping_cd = (23 + 2.21)
        crowdshipping_vehicles = int(0)
        km_crowdshipping_daily_total = float(0)
        crowdshipping_deliveries_daily_total = int(0)
        

                                    # parcel locker
        km_parcelocker = float(0)
        parcelocker_deliveries = int(0)
        parcelocker_cd = float(0)

        vehicles_deliveries = int(0)
        vehicles = int(0)

        packages = []
        km = []
        cd = []
        dist_clients = []
        dist_dc = []
        orders = []
        deliveries = int(0)
        total_packages = int(0)
        total_mileage = int(0)
        tw_count = []

        for i in range(len(tw)):
            orders.append(int(0))

                                # Create Orders

        for i in range(resp1):  # Parcel distribuition

            ed = monte_carlo(input_csv_mix())  # gen_delivery
            tw[ed] = tw[ed] + 1
            orders[ed] = orders[ed] + 1

        for i in range(len(tw)):
            tw_count.append(tw[i])
            
                                # Modality Distribuition by percentage
                
        vehicle_deliveries = np.floor((tw[0] * percentage_traditional))
        orders_crowdshipping = np.floor((tw[0] * (percentage_crowdshipping)))
        orders_parcelocker = np.ceil((tw[0] * (percentage_parcel_locker)))
                                
                                # If there is one parcel remaining 
                
        if tw[0] - (vehicle_deliveries + orders_crowdshipping + orders_parcelocker) > 0:
            orders_parcelocker = orders_parcelocker + (
                        tw[0] != (vehicle_deliveries + orders_crowdshipping + orders_parcelocker))

        if vehicle_deliveries > 0:

            if (vehicle_deliveries) % cl > 0:
                vehicles = int(abs(((vehicle_deliveries) // cl) + 1))
            else:
                vehicles = int(abs((vehicle_deliveries) / cl))

        if vehicles > max_vehicles and day_counter > warmup:
            max_vehicles = vehicles

        if vehicles < min_vehicles and vehicles != 0 and day_counter > warmup:
            min_vehicles = vehicles

                                # Append the lists

        for i in range((vehicles)):  # defines the array of packages of a car

            km.append(int(0))
            cd.append(int(0))
            dist_clients.append(int(0))
            dist_dc.append(int(0))

                                # Manage the packages

        if vehicles > 0:
            for i in range(vehicles):
                packages.append(int(0))

                while (vehicle_deliveries) > 0 and packages[i] < int(cl):
                    vehicle_deliveries -= 1
                    tw[0] -= 1
                    packages[i] += 1
                    total_packages += 1
                    if day_counter > (warmup):  # Saves delivery only after warmup days
                        overall_packages += 1

                    deliveries += 1

                            # Calculate the Mileage

            for b in range(int(vehicles)):
                dist_clients[b] = 2.21 * ((packages[b]) - 1)  # Defines the distance between clients
                dist_dc[b] = 23 * 2  # Defines the distance between Distribuition Center and residential area

                km[b] = dist_clients[b] + dist_dc[b]  # Defines the mileage

                total_mileage = total_mileage + km[b]

            if day_counter > (warmup):  # Saves mileage only after 13 days
                overall_mileage = overall_mileage + total_mileage  # Defines the overall km
                rep_km.append(overall_mileage)

                            # Crowdshipping

        if (int(orders_crowdshipping)) % 2 > 0:
            crowdshipping_vehicles = int(abs(((orders_crowdshipping) // 2) + 1))
        else:
            crowdshipping_vehicles = int(abs((orders_crowdshipping) / 2))

        if day_counter > warmup:
            total_crowdshipping_vehicles += crowdshipping_vehicles

        for f in range(crowdshipping_vehicles):  # Defines how many crowdshipping vehicles
            crowdshipping_deliveries.append(0)
            km_crowdshipping.append(0)

        for f in range(crowdshipping_vehicles):

            while (orders_crowdshipping > 0) and (crowdshipping_deliveries[f] < 2):  # crowdshipping parcels
                crowdshipping_deliveries[f] = crowdshipping_deliveries[f] + 1
                orders_crowdshipping -= 1

                if day_counter > warmup:
                    crowdshipping_deliveries_total += 1

                tw[0] = tw[0] - 1

            if (crowdshipping_deliveries[f]) == 2:
                km_crowdshipping[f] = km_crowdshipping[f] + crowdshipping_cd
                if day_counter > warmup:
                    km_crowdshipping_total = km_crowdshipping_total + km_crowdshipping[f]

            if (crowdshipping_deliveries[f]) == 1:
                km_crowdshipping[f] = km_crowdshipping[f] + (crowdshipping_cd - 2.21)
                if day_counter > warmup:
                    km_crowdshipping_total = km_crowdshipping_total + km_crowdshipping[f]
            
            
        for crowd in range (crowdshipping_vehicles):
            km_crowdshipping_daily_total += km_crowdshipping[crowd]
            crowdshipping_deliveries_daily_total += crowdshipping_deliveries[crowd]
            
                

                            # Parcelocker

        parcelocker_cd = (23 + 23 + (1.5 * 12))

        if day_counter > warmup:
            km_parcelocker += parcelocker_cd
            km_parcelocker_total += parcelocker_cd
            parcelocker_deliveries += orders_parcelocker
            parcelocker_deliveries_total += orders_parcelocker

        tw[0] = tw[0] - orders_parcelocker

                            # Adjust the Time Window

        tw[0] = tw[0] + tw[1]

        for i in range((len(tw) - 2)):
            tw[i + 1] = tw[i + 2]

        tw[24] = 0

        if mean_results == "on":
            mean_km = float(0)
            mean_km = overall_mileage + km_crowdshipping_total + km_parcelocker
            replication_km.append(mean_km)
            del mean_km

#=================================================================== DEBUG            

        if debug == "on":
            print('█████████████████████████dia {}█████████████████████████'.format(day_counter))
            print('\nEstoque inicio do dia: {}'.format(tw_count))
            print('estoque fim do dia: {}'.format(tw))

            print('\n=============================== PEDIDOS TRADICIONAIS ===============================')

            print('\nentregas: {}'.format(deliveries))

            print('veiculos: {}'.format(vehicles))

            print('km: {}'.format(km))
            print('quilometragem: {:.2f}'.format(overall_mileage))
            print('entregas totais: {}'.format(overall_packages))

            print('\n=============================== PEDIDOS CROWDSHIPPING ===============================')
            print('kilometragem: {}'.format(km_crowdshipping))
            print('kilometragem total: {}'.format(km_crowdshipping_daily_total))
            print('entregas: {}'.format(crowdshipping_deliveries))
            print('entregas totais: {}'.format(crowdshipping_deliveries_daily_total))
            print('\n')

            print('\n=============================== PEDIDOS PARCEL LOCKER ===============================')
            print('kilometragem: {}'.format(km_parcelocker))
            print('entregas: {}'.format(parcelocker_deliveries))
            print('\n')

#=================================================================== DELETES THE PARAMETERS

        del vehicles
        del packages
        del km
        del cd
        del dist_clients
        del dist_dc
        del deliveries
        del total_packages
        del total_mileage
        del tw_count
        del orders
        del vehicles_deliveries
        

                                # Crowdshipping

        del km_crowdshipping
        del km_crowdshipping_daily_total
        del crowdshipping_deliveries
        del crowdshipping_deliveries_daily_total
        del crowdshipping_cd
        del crowdshipping_vehicles

                                # Parcel Locker
        del km_parcelocker
        del parcelocker_deliveries
        del parcelocker_cd
        del orders_parcelocker

        days = days - 1
        day_counter += 1

#=================================================================== RESULTS SCREEN

    if results == "on":
        print('\n===')
        print('\nReplicação {}'.format(l + 1))
        print('Simulação com {} dias, consolidação de {} '.format(total_days, cl))
        print('===')
        print('\n=============================== PEDIDOS TRADICIONAIS ===============================')
        print('max veiculos tradicional: {}'.format(max_vehicles))
        print('min veiculos tradicional: {}'.format(min_vehicles))
        print('quilometragem tradicional: {:.2f}'.format(overall_mileage))
        print('entregas totais tradicional: {}\n'.format(overall_packages))
        print('\n=============================== PEDIDOS CROWDSHIPPING ===============================')
        print('quilometragem total crowdshipping: {:.2f}\n'.format(km_crowdshipping_total))
        print('entregas totais crowdshipping: {}'.format(crowdshipping_deliveries_total))
        print('Veiculos  utilizados crowdshipping: {}'.format(total_crowdshipping_vehicles))
        print('\n=============================== PEDIDOS PARCEL LOCKER ===============================')
        print('entregas totais parcelocker: {}'.format(parcelocker_deliveries_total))
        print('quilometragem total parcelocker:{}\n'.format(km_parcelocker_total))

    # APPEND THE MEANS LIST
    

    if runs > 0:
        day_counter = int(1)
        days = total_days
        overall_mileage = int(0)
        overall_packages = int(0)

#=================================================================== RESETS THE REPLICATIONS PARAMETERS
    del km_parcelocker_total
    del parcelocker_deliveries_total
    del total_crowdshipping_vehicles
    del km_crowdshipping_total
    del crowdshipping_deliveries_total
    del total_days
    del cl
    del tw
    del max_vehicles
    del min_vehicles
    del day_counter
    del overall_mileage
    del overall_packages

    data = replication_km




i = 0

if mean_results == "on":
    for i in range(runs):
        rep_km.append(np.mean(replication_km[i]))
    print('\n=============================== MEAN TEST ===============================')    
    print("{} replications".format(runs))
    print("km mean:")
    print(np.mean(rep_km))
    print("km standard deviation:")
    print(np.std(rep_km))
    print("km mean and confidence_interval and h")
    print(mean_confidence_interval_and_h(rep_km))
    print("---------------------------------")
    
   


===

Replicação 1
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 5
min veiculos tradicional: 3
quilometragem tradicional: 20244.50
entregas totais tradicional: 2582


=============================== PEDIDOS CROWDSHIPPING ===============================
quilometragem total crowdshipping: 22009.72

entregas totais crowdshipping: 1709
Veiculos  utilizados crowdshipping: 877

=============================== PEDIDOS PARCEL LOCKER ===============================
entregas totais parcelocker: 4447.0
quilometragem total parcelocker:5760.0


===

Replicação 2
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 4
min veiculos tradicional: 3
quilometragem tradicional: 19839.34
entregas totais tradicional: 2577


=============================== PEDIDOS CROWDSHIPPING ================


===

Replicação 14
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 4
min veiculos tradicional: 3
quilometragem tradicional: 20133.43
entregas totais tradicional: 2611


=============================== PEDIDOS CROWDSHIPPING ===============================
quilometragem total crowdshipping: 22397.61

entregas totais crowdshipping: 1734
Veiculos  utilizados crowdshipping: 893

=============================== PEDIDOS PARCEL LOCKER ===============================
entregas totais parcelocker: 4495.0
quilometragem total parcelocker:5760.0


===

Replicação 15
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 4
min veiculos tradicional: 3
quilometragem tradicional: 20428.91
entregas totais tradicional: 2606


=============================== PEDIDOS CROWDSHIPPING ==============


===

Replicação 27
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 5
min veiculos tradicional: 3
quilometragem tradicional: 20172.39
entregas totais tradicional: 2589


=============================== PEDIDOS CROWDSHIPPING ===============================
quilometragem total crowdshipping: 22135.77

entregas totais crowdshipping: 1719
Veiculos  utilizados crowdshipping: 882

=============================== PEDIDOS PARCEL LOCKER ===============================
entregas totais parcelocker: 4461.0
quilometragem total parcelocker:5760.0


===

Replicação 28
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 4
min veiculos tradicional: 3
quilometragem tradicional: 20003.45
entregas totais tradicional: 2572


=============================== PEDIDOS CROWDSHIPPING ==============


===

Replicação 40
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 4
min veiculos tradicional: 3
quilometragem tradicional: 20310.39
entregas totais tradicional: 2592


=============================== PEDIDOS CROWDSHIPPING ===============================
quilometragem total crowdshipping: 22071.19

entregas totais crowdshipping: 1718
Veiculos  utilizados crowdshipping: 879

=============================== PEDIDOS PARCEL LOCKER ===============================
entregas totais parcelocker: 4459.0
quilometragem total parcelocker:5760.0


===

Replicação 41
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 4
min veiculos tradicional: 3
quilometragem tradicional: 20124.18
entregas totais tradicional: 2587


=============================== PEDIDOS CROWDSHIPPING ==============


===

Replicação 53
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 5
min veiculos tradicional: 3
quilometragem tradicional: 20029.97
entregas totais tradicional: 2584


=============================== PEDIDOS CROWDSHIPPING ===============================
quilometragem total crowdshipping: 22085.35

entregas totais crowdshipping: 1715
Veiculos  utilizados crowdshipping: 880

=============================== PEDIDOS PARCEL LOCKER ===============================
entregas totais parcelocker: 4446.0
quilometragem total parcelocker:5760.0


===

Replicação 54
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 5
min veiculos tradicional: 3
quilometragem tradicional: 20347.96
entregas totais tradicional: 2609


=============================== PEDIDOS CROWDSHIPPING ==============

...


===

Replicação 469
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 5
min veiculos tradicional: 3
quilometragem tradicional: 20371.86
entregas totais tradicional: 2600


=============================== PEDIDOS CROWDSHIPPING ===============================
quilometragem total crowdshipping: 22190.61

entregas totais crowdshipping: 1725
Veiculos  utilizados crowdshipping: 884

=============================== PEDIDOS PARCEL LOCKER ===============================
entregas totais parcelocker: 4487.0
quilometragem total parcelocker:5760.0


===

Replicação 470
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 5
min veiculos tradicional: 3
quilometragem tradicional: 20060.50
entregas totais tradicional: 2578


=============================== PEDIDOS CROWDSHIPPING ============


===

Replicação 482
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 5
min veiculos tradicional: 3
quilometragem tradicional: 20075.97
entregas totais tradicional: 2585


=============================== PEDIDOS CROWDSHIPPING ===============================
quilometragem total crowdshipping: 22074.30

entregas totais crowdshipping: 1710
Veiculos  utilizados crowdshipping: 880

=============================== PEDIDOS PARCEL LOCKER ===============================
entregas totais parcelocker: 4452.0
quilometragem total parcelocker:5760.0


===

Replicação 483
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 4
min veiculos tradicional: 3
quilometragem tradicional: 20174.60
entregas totais tradicional: 2590


=============================== PEDIDOS CROWDSHIPPING ============


===

Replicação 495
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 4
min veiculos tradicional: 3
quilometragem tradicional: 19880.92
entregas totais tradicional: 2576


=============================== PEDIDOS CROWDSHIPPING ===============================
quilometragem total crowdshipping: 21959.30

entregas totais crowdshipping: 1705
Veiculos  utilizados crowdshipping: 875

=============================== PEDIDOS PARCEL LOCKER ===============================
entregas totais parcelocker: 4430.0
quilometragem total parcelocker:5760.0


===

Replicação 496
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 4
min veiculos tradicional: 3
quilometragem tradicional: 20367.44
entregas totais tradicional: 2598


=============================== PEDIDOS CROWDSHIPPING ============


===

Replicação 508
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 4
min veiculos tradicional: 3
quilometragem tradicional: 20115.34
entregas totais tradicional: 2583


=============================== PEDIDOS CROWDSHIPPING ===============================
quilometragem total crowdshipping: 22118.09

entregas totais crowdshipping: 1711
Veiculos  utilizados crowdshipping: 882

=============================== PEDIDOS PARCEL LOCKER ===============================
entregas totais parcelocker: 4453.0
quilometragem total parcelocker:5760.0


===

Replicação 509
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 4
min veiculos tradicional: 3
quilometragem tradicional: 20087.02
entregas totais tradicional: 2590


=============================== PEDIDOS CROWDSHIPPING ============


===

Replicação 521
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 4
min veiculos tradicional: 3
quilometragem tradicional: 20001.65
entregas totais tradicional: 2591


=============================== PEDIDOS CROWDSHIPPING ===============================
quilometragem total crowdshipping: 22117.19

entregas totais crowdshipping: 1720
Veiculos  utilizados crowdshipping: 881

=============================== PEDIDOS PARCEL LOCKER ===============================
entregas totais parcelocker: 4453.0
quilometragem total parcelocker:5760.0


===

Replicação 522
Simulação com 120 dias, consolidação de 9 
===

=============================== PEDIDOS TRADICIONAIS ===============================
max veiculos tradicional: 5
min veiculos tradicional: 3
quilometragem tradicional: 20393.96
entregas totais tradicional: 2610


=============================== PEDIDOS CROWDSHIPPING ============